In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/tornikeo/Documents/uni/PRCV/proj


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests
from pathlib import Path
import random
import re

In [6]:
from utils import load_mnist, load_training_data

d_x, d_y = load_training_data()
m_x, m_y = load_mnist()

In [7]:
from utils import load_rasterized_data, show

data_x, data_y = load_rasterized_data()

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [116]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import torch

# Split the LM data into 50/50
# Add MNIST to training half, test with only LM held-out dataset.
x_train, x_test, y_train, y_test = train_test_split(
    data_x, 
    data_y, 
    test_size=0.50, 
    shuffle=True,
    stratify=data_y,
    random_state=42,
)

x_train = np.concatenate((m_x, x_train)) 
y_train = np.concatenate((m_y, y_train))

x_train = x_train / 255
x_test = x_test / 255

# x_train = x_train.reshape(len(x_train), -1)
# x_test = x_test.reshape(len(x_test), -1)

x_train = torch.tensor(x_train).float().unsqueeze(1)
x_train_lm = x_train[-500:]
x_test = torch.tensor(x_test).float().unsqueeze(1)

y_train = torch.tensor(y_train)
y_train_lm = y_train[-500:]
y_test = torch.tensor(y_test)

In [157]:
from torch import nn
from torch.utils import data
from torch.nn import functional as F

train_ds = data.TensorDataset(x_train, y_train)
test_ds = data.TensorDataset(x_test, y_test)

train_dl = data.DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = data.DataLoader(test_ds, batch_size=64, shuffle=True)

# train_main_dl


class Net(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv = nn.Conv2d(1, 32, (3,3),)
        self.pool = nn.MaxPool2d((2,2))
        # self.bn = nn.BatchNorm2d(32)
        self.flat = nn.Flatten()
        self.ff = nn.Linear(5408, 100)
        self.out = nn.Linear(100, 10)
    def forward(self, x):
        x = F.leaky_relu(self.conv(x))
        x = self.pool(x)
        # x = self.bn(x)
        x = self.flat(x)
        # print(x.shape)
        x = F.leaky_relu(self.ff(x))
        x = self.out(x)
        return x
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Net()
optim = torch.optim.SGD(model.parameters(), lr=0.01, momentum=.9)
model.to(device)
for ep in range(10):
    for bn, (xb, yb) in enumerate(train_dl):
        xb = xb.to(device)
        yb = yb.to(device)
        ll = model(xb)
        loss = F.nll_loss(F.log_softmax(ll, dim=1), yb)
        loss.backward()
        optim.step()
        optim.zero_grad()
    print("MNIST LOSS", loss.item(), end='\r')
print()


for _ in range(10): 
    optim = torch.optim.SGD(model.parameters(), lr=0.01, momentum=.9)
    x_train_lm = x_train_lm.to(device)
    y_train_lm = y_train_lm.to(device)
    for _ in range(100):
        ll = model(x_train_lm)
        loss = F.nll_loss(F.log_softmax(ll, dim=1), y_train_lm)
        loss.backward()
        optim.step()
        optim.zero_grad()
        # We are only interested in loss over LM data
        print("LeapMotion LOSS", loss.item(), end='\r')
    print()

    with torch.no_grad():
        
        model.eval()
        x_test = x_test.to(device)
        y_test = y_test.to(device)
        
        ll = model(x_test)
        proba = F.log_softmax(ll, dim=1)
        loss = F.nll_loss(proba, y_test)
        predicted = proba.argmax(-1)
        
        print(classification_report(y_test.cpu(), predicted.cpu()))
        print(confusion_matrix(y_test.cpu(), predicted.cpu()))

MNIST LOSS 0.001512818387709558
LeapMotion LOSS 0.0044087427668273455
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        50
           1       0.95      0.80      0.87        50
           2       0.98      1.00      0.99        50
           3       0.98      0.96      0.97        50
           4       0.94      0.94      0.94        50
           5       0.92      0.90      0.91        50
           6       0.90      0.92      0.91        50
           7       0.84      0.98      0.91        50
           8       0.98      0.90      0.94        50
           9       0.92      0.98      0.95        50

    accuracy                           0.94       500
   macro avg       0.94      0.94      0.94       500
weighted avg       0.94      0.94      0.94       500

[[49  0  0  0  0  0  0  0  1  0]
 [ 0 40  0  0  2  1  0  5  0  2]
 [ 0  0 50  0  0  0  0  0  0  0]
 [ 0  1  0 48  0  1  0  0  0  0]
 [ 0  0  0  0 47  1  0  2  0  0]
 [ 0  

              precision    recall  f1-score   support

           0       0.92      0.94      0.93        50
           1       1.00      0.66      0.80        50
           2       0.47      1.00      0.64        50
           3       0.61      0.92      0.74        50
           4       0.96      0.88      0.92        50
           5       0.44      0.64      0.52        50
           6       0.80      0.08      0.15        50
           7       0.66      0.98      0.79        50
           8       0.56      0.10      0.17        50
           9       0.90      0.52      0.66        50

    accuracy                           0.67       500
   macro avg       0.73      0.67      0.63       500
weighted avg       0.73      0.67      0.63       500

1.1630265712738037, 

In [62]:
xb

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [50]:
# import scipy
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


# parameters = {'n_neighbors':range(1,10), 'p':[1,2]}
# svc = KNeighborsClassifier()
# clf = RandomizedSearchCV(svc, parameters, cv=2, n_jobs=-1)
# clf.fit(x_train, y_train)
# print(clf.best_params_)

# print(classification_report(y_test, clf.predict(x_test)))
# print(confusion_matrix(y_test, clf.predict(x_test)))

KeyboardInterrupt: 

{'n_neighbors': 1, 'p': 2}